# Fact checking

## Use case

`Fact-checking` is a technique to address `hallucination`. It verifies claims made by LLMs. Usually, it verifies claims against evidence from external sources.

## Overview

Usually, fact-checking has these steps:
- **Claim detection**, called also the **Fact detection**, identifies claims or facts in the LLM prime answer
- **Claim verification** evaluates each claim and decides to support or reject the claim
- **Final decision** based on all claim verification results decides on the result of the whole answer

## Quickstart

### Setting up

In [1]:
#!pip install langchain langchain-openai

## Set env var OPENAI_API_KEY or load from a .env file:

import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("OPENAI_API_KEY = ")

# import dotenv
# dotenv.load_dotenv()

OPENAI_API_KEY =  ········


In [5]:
## setup LangSmith tracing:

import os
from getpass import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "True"
os.environ["LANGCHAIN_API_KEY"] = getpass("LANGCHAIN_API_KEY = ")

LANGCHAIN_API_KEY =  ········


Now, we are ready to initiate the chain:

In [10]:
from langchain.chains import LLMCheckerChain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0.7)
checker_chain = LLMCheckerChain.from_llm(llm, verbose=True)

### Verify question

We can run verification together with questioning LLM with no changes in the original prompt.

In [6]:
text = "What type of mammal lays the biggest eggs?"
checker_chain.invoke(text)



> Entering new LLMCheckerChain chain...


> Entering new SequentialChain chain...

> Finished chain.

> Finished chain.


{'query': 'What type of mammal lays the biggest eggs?',
 'result': 'Monotremes are the type of mammal that lays the biggest eggs.'}

Then we can look at the `LangSmith` trace to see what is happening under the hood:

Summary: 
Here we see the result of the chain. There were 4 calls to LLM. Number of calls for the chain is fixed.

![image.png](../../static/img/fakt_checking.example_1.1.png)

1. It is the preliminary step. LLM receives the original prompt and generates an answer.

![image.png](../../static/img/fakt_checking.example_1.2.png)

2. It is the `Claim detection` step. The chain adds this text to the result of the first call: `Make a bullet point list of the assumptions you made when producing the above statement.` The result is the list of the claims/facts.
   
![image.png](../../static/img/fakt_checking.example_1.3.png)

3. It is the `Claim verification` step. Now the chain adds this text to the result of the second call: `For each assertion, determine whether it is true or false. If it is false, explain why.` LLM verifies each claim and decides if it is correct or not by adding True or False to each claim.

![image.png](../../static/img/fakt_checking.example_1.4.png)

4. It is the `Final decision` step. The chain adds this text to the result of the third call: `Question: In light of the above assertions and checks, how would you answer the question '<Original prompt>'?` LLM produces the final result text.

![image.png](../../static/img/fakt_checking.example_1.5.png)

The final result is a little bit different from the first call result. The first result provides us with two mammals but the final result provides just a single mammal. 
This difference could be important in your specific use case.

### Explicit verification

We can explicitly ask for verification in the existing text by, for example, `Is it correct?` question.

The text in this example holds incorrect factual information. The chain finds that the result is wrong and provides the correct fact.

In [9]:
text = "Vancouver was established in 1786. Is it correct?"
checker_chain.invoke(text)



> Entering new LLMCheckerChain chain...


> Entering new SequentialChain chain...

> Finished chain.

> Finished chain.


{'query': 'Vancouver was established in 1786. Is it correct?',
 'result': 'No, it is not correct. Vancouver was not established in 1786, it was established in 1886.'}

Here we provide only the input and output of each call:

![image.png](../../static/img/fakt_checking.example_2.1.png)

![image.png](../../static/img/fakt_checking.example_2.2.png)

![image.png](../../static/img/fakt_checking.example_2.3.png)

![image.png](../../static/img/fakt_checking.example_2.4.png)
 

### Fact checking

We can just run any verification on any text with presumable facts inside.

This example is similar to the previous one but does not ask explicitly to verify the text. 
Still, the chain successfully finds the wrong fact and provides us with the correct information.

Note, that the result text is not the same as in the previous example.

In [8]:
text = "Vancouver was established in 1786."
checker_chain.invoke(text)



> Entering new LLMCheckerChain chain...


> Entering new SequentialChain chain...

> Finished chain.

> Finished chain.


{'query': 'Vancouver was established in 1786.',
 'result': 'False. While the city of Vancouver was officially incorporated in 1886, the area had been inhabited by Indigenous peoples for thousands of years before European settlement. The statement that Vancouver was established in 1786 is not accurate.'}

Here we provide only the input and output of each call:

![image.png](../../static/img/fakt_checking.example_3.1.png)

![image.png](../../static/img/fakt_checking.example_3.2.png)

![image.png](../../static/img/fakt_checking.example_3.3.png)

![image.png](../../static/img/fakt_checking.example_3.4.png)
 

**Note:**

- the first result immediately points out that the fact in the prompt was not correct.
- the final result is not the same as the first result